## Data Understanding and Preparation
<img src="https://github.com/CatherineCao2016/lendingclub/raw/master/cleaning.png" width="800" height="500" align="middle"/>

There are a number of fields that are populated with a large number of NA values and also a number of fields of data that were filled after the loan was given - since we are modeling whether a loan should be given or not we are only concerned with the information provided to Lending Club when the loan was requested. So we are going to do a first pass through the data and select out only the variables we are interested in keeping.

* Outcome: Loan Status
* Loan application info: 
    * Issue date, 
    * loan amount
    * employment title
    * employment length
    * verification status
    * home ownership
    * annual income
    * purpose, 
    * loan description
    * address
    * term

* Background check: 
    * financial inquiries in last 6 months
    * open credit lines, 
    * derogatory public records
    * Revolving line utilization rate
    * debt-to-income ratio
    * total credit lines
    * delinquency instances in past 2 years
    * earliest reported credit line open time
    * Months since last delinquency

* Computed additional features: 
    * EMP_LISTED
    * EMPTY_DESC
    * EMP_NA,
    * DELING_EVER
    * TIME_HISTORY

## Import Libraries

In [1]:
import ibmdbpy
from ibmdbpy import IdaDataBase, IdaDataFrame
import pandas as pd
pd.options.display.max_columns = 999
import warnings
warnings.filterwarnings('ignore')
import time
from datetime import datetime
import math

from pyspark.sql import SparkSession
from pyspark.sql.types import DoubleType


## Load Sample Data 
### The lending club data set has been preloaded into dashdb

In [ ]:
# The code was removed by DSX for sharing.

In [ ]:
# Credentials for DashDB hidden
# Extract first 40000 records into a Pandas Dataframe

loan = IdaDataFrame(idadb_conn, 'DASH111207.LOAN40000').as_dataframe()

## Quick Overview

In [3]:
print "There are " + str(len(loan)) + " observations in the dataset."
print "There are " + str(len(loan.columns)) + " variables in the dataset."

print "\n******************Dataset Quick View*****************************\n"
loan.head(5)

There are 39999 observations in the dataset.
There are 74 variables in the dataset.

******************Dataset Quick View*****************************



,ID,MEMBER_ID,LOAN_AMNT,FUNDED_AMNT,FUNDED_AMNT_INV,TERM,INT_RATE,INSTALLMENT,GRADE,SUB_GRADE,EMP_TITLE,EMP_LENGTH,HOME_OWNERSHIP,ANNUAL_INC,VERIFICATION_STATUS,ISSUE_D,LOAN_STATUS,PYMNT_PLAN,URL,DESC,PURPOSE,TITLE,ZIP_CODE,ADDR_STATE,DTI,DELINQ_2YRS,EARLIEST_CR_LINE,INQ_LAST_6MTHS,MTHS_SINCE_LAST_DELINQ,MTHS_SINCE_LAST_RECORD,OPEN_ACC,PUB_REC,REVOL_BAL,REVOL_UTIL,TOTAL_ACC,INITIAL_LIST_STATUS,OUT_PRNCP,OUT_PRNCP_INV,TOTAL_PYMNT,TOTAL_PYMNT_INV,TOTAL_REC_PRNCP,TOTAL_REC_INT,TOTAL_REC_LATE_FEE,RECOVERIES,COLLECTION_RECOVERY_FEE,LAST_PYMNT_D,LAST_PYMNT_AMNT,NEXT_PYMNT_D,LAST_CREDIT_PULL_D,COLLECTIONS_12_MTHS_EX_MED,MTHS_SINCE_LAST_MAJOR_DEROG,POLICY_CODE,APPLICATION_TYPE,ANNUAL_INC_JOINT,DTI_JOINT,VERIFICATION_STATUS_JOINT,ACC_NOW_DELINQ,TOT_COLL_AMT,TOT_CUR_BAL,OPEN_ACC_6M,OPEN_IL_6M,OPEN_IL_12M,OPEN_IL_24M,MTHS_SINCE_RCNT_IL,TOTAL_BAL_IL,IL_UTIL,OPEN_RV_12M,OPEN_RV_24M,MAX_BAL_BC,ALL_UTIL,TOTAL_REV_HI_LIM,INQ_FI,TOTAL_CU_TL,INQ_LAST_12M
0,849642,1061376,4500,4500,4475,36 months,5.42,135.72,A,A1,The Designory,< 1 year,RENT,67000,Source Verified,Aug-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 08/24/11 > Purchasing a moto...,major_purchase,Motorcycle Loan,908xx,CA,5.39,0,Oct-1994,1,NaN,NaN,9,0,6570,18.3,16,f,0,0,4884.504294,4857.37,4500.00,384.50,0,0.00,0.00,Sep-2014,149.09,None,Nov-2015,0,None,1,INDIVIDUAL,None,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,856538,1068909,10000,10000,10000,36 months,9.99,322.63,B,B1,Axway Inc,7 years,RENT,95000,Not Verified,Aug-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 08/20/11 > I never missed a ...,debt_consolidation,Debt Consolidation,852xx,AZ,3.75,0,Jul-2005,1,NaN,NaN,2,0,2220,26.4,5,f,0,0,11030.820000,11030.82,10000.00,1030.82,0,0.00,0.00,Jul-2013,290.38,None,Jan-2016,0,None,1,INDIVIDUAL,None,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,856531,1068900,14500,14500,14500,36 months,5.42,437.32,A,A1,Dch Chrysler Jeep Dodge of Temecula,10+ years,MORTGAGE,70000,Verified,Aug-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,None,credit_card,Reduction of interest,925xx,CA,8.73,0,Apr-1994,0,NaN,NaN,8,0,66003,11.4,25,f,0,0,7471.480000,7471.48,6150.73,841.43,0,479.32,4.88,Jan-2013,437.32,None,May-2013,0,None,1,INDIVIDUAL,None,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,850095,1061880,5200,5200,5200,60 months,11.49,114.34,B,B4,USPS,1 year,MORTGAGE,57000,Verified,Aug-2011,Charged Off,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 08/19/11 > Thank you very ...,home_improvement,Home Improvement,921xx,CA,0.32,0,Feb-1997,2,33,NaN,7,0,889,7.5,16,f,0,0,1708.480000,1708.48,808.76,634.83,0,264.89,2.26,Nov-2012,200.00,None,Feb-2013,0,None,1,INDIVIDUAL,None,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,851229,1063133,5000,5000,5000,36 months,5.42,150.80,A,A1,Kansas University Medical Center,5 years,MORTGAGE,65000,Not Verified,Aug-2011,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail....,Borrower added on 08/23/11 > - Loan is to re-f...,major_purchase,Ford F150 Truck,640xx,MO,4.56,0,Jul-1998,1,NaN,NaN,7,0,3856,13.7,18,f,0,0,5376.020000,5376.02,5000.00,376.02,0,0.00,0.00,Sep-2013,1921.03,None,Aug-2013,0,None,1,INDIVIDUAL,None,None,None,0,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


### Descriptive Statistics

In [4]:
print "\n******************Descriptive statistics*****************************\n"
# Note this table only shows variables that are non categorical .. ie numerical
loan.describe()


******************Descriptive statistics*****************************



,ID,MEMBER_ID,LOAN_AMNT,FUNDED_AMNT,FUNDED_AMNT_INV,INT_RATE,INSTALLMENT,ANNUAL_INC,DTI,DELINQ_2YRS,INQ_LAST_6MTHS,MTHS_SINCE_LAST_DELINQ,MTHS_SINCE_LAST_RECORD,OPEN_ACC,PUB_REC,REVOL_BAL,REVOL_UTIL,TOTAL_ACC,OUT_PRNCP,OUT_PRNCP_INV,TOTAL_PYMNT,TOTAL_PYMNT_INV,TOTAL_REC_PRNCP,TOTAL_REC_INT,TOTAL_REC_LATE_FEE,RECOVERIES,COLLECTION_RECOVERY_FEE,LAST_PYMNT_AMNT,COLLECTIONS_12_MTHS_EX_MED,POLICY_CODE,ACC_NOW_DELINQ
count,39999.000000,39999.000000,39999.00000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,14158.000000,2815.000000,39999.000000,39999.000000,39999.000000,39949.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39943,39999,39999
mean,683032.262282,850464.777969,11220.38176,10947.754944,10399.141609,12.039807,324.402967,69005.632504,13.328588,0.147404,0.889347,35.919551,69.917229,9.304558,0.055451,13410.941999,48.874830,22.113228,136.416712,135.621318,12068.393933,11484.253011,9710.929387,2262.598155,1.358577,93.507826,12.164093,2659.074573,0,1,0
std,210175.718857,264969.606267,7458.32188,7188.124706,7128.423563,3.728041,208.824838,63903.736916,6.680936,0.495918,1.108814,22.016964,43.703928,4.414575,0.238177,15960.968973,28.331199,11.419090,750.905477,747.620254,8906.812986,8804.453726,6974.610134,2601.845374,7.229968,683.301966,149.191796,4441.776370,0,0,0
min,54734.000000,70699.000000,500.00000,500.000000,0.000000,5.420000,15.690000,4000.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,1,0
25%,517089.000000,667744.500000,5500.00000,5400.000000,5000.000000,9.250000,166.800000,40500.000000,8.190000,0.000000,0.000000,18.000000,24.000000,6.000000,0.000000,3700.500000,25.500000,14.000000,0.000000,0.000000,5565.580000,5092.615000,4518.950000,661.320000,0.000000,0.000000,0.000000,218.075000,0,1,0
50%,664395.000000,849289.000000,10000.00000,9600.000000,8975.000000,11.860000,280.010000,59000.000000,13.420000,0.000000,1.000000,34.000000,90.000000,9.000000,0.000000,8858.000000,49.300000,20.000000,0.000000,0.000000,9876.190000,9274.460000,8000.000000,1348.930000,0.000000,0.000000,0.000000,533.750000,0,1,0
75%,837083.000000,1046704.500000,15000.00000,15000.000000,14400.000000,14.590000,430.750000,82400.000000,18.620000,0.000000,1.000000,52.000000,104.000000,12.000000,0.000000,17065.500000,72.400000,29.000000,0.000000,0.000000,16444.420000,15711.905000,13424.260000,2835.770000,0.000000,0.000000,0.000000,3246.385000,0,1,0
max,1077501.000000,1314167.000000,35000.00000,35000.000000,35000.000000,24.590000,1305.190000,6000000.000000,29.990000,11.000000,8.000000,120.000000,129.000000,44.000000,4.000000,149588.000000,99.900000,90.000000,10418.060000,10410.560000,56809.051629,56475.050000,35000.030000,23062.450000,166.429711,29623.350000,7002.190000,36115.200000,0,1,0


### 1. Keep Variables of Interest

In [5]:
keep_list = ['LOAN_STATUS', 'ISSUE_D', 'LOAN_AMNT', 'EMP_TITLE', 'EMP_LENGTH', 'VERIFICATION_STATUS', 'HOME_OWNERSHIP', 'ANNUAL_INC', 'PURPOSE', 'INQ_LAST_6MTHS', 'DESC', 'OPEN_ACC', 'PUB_REC', 'REVOL_UTIL', 'DTI', 'TOTAL_ACC', 'DELINQ_2YRS','EARLIEST_CR_LINE', 'MTHS_SINCE_LAST_DELINQ', 'ADDR_STATE', 'TERM']
loan_sub_kp = loan[keep_list]

### 2. Encode dependent variable from loan_status 

In [6]:
# use a lamba function to encode multiple loan_status entries into a single 1/0 default variable
loan_sub_kp['DEFAULT'] = loan_sub_kp['LOAN_STATUS'].isin([
    'Default',
    'Charged Off',
    'Late (31-120 days)',
    'Late (16-30 days)',
    'Does not meet the credit policy. Status:Charged Off'
]).map(lambda x: int(x))

### 3. Handle Missing Values

In [7]:
# Replacing missing values with 0's
loan_sub_kp['EMP_LISTED'] = [1 if x != None else 0 for x in loan_sub_kp['EMP_TITLE']]
loan_sub_kp['EMPTY_DESC'] = [1 if x == None else 0 for x in loan_sub_kp['DESC']]
loan_sub_kp['EMP_NA'] = [1 if x == "n/a" else 0 for x in loan_sub_kp['EMP_LENGTH']]
loan_sub_kp['EMP_LENGTH'] = ['Other' if x == "n/a" else x for x in loan_sub_kp['EMP_LENGTH']]
loan_sub_kp['DELING_EVER'] = [0 if math.isnan(x) else 1 for x in loan_sub_kp['MTHS_SINCE_LAST_DELINQ']]
loan_sub_kp['HOME_OWNERSHIP'] = ["OTHER" if x == None else x for x in loan_sub_kp['HOME_OWNERSHIP'] ]

### 4. Handle Time Objects

In [8]:
loan_sub_kp['ISSUE_D'] = loan_sub_kp['ISSUE_D'].map(lambda x: datetime.strptime(x, "%b-%Y"))
loan_sub_kp['EARLIEST_CR_LINE'] = loan_sub_kp['EARLIEST_CR_LINE'].map(lambda x: datetime.strptime(x, '%b-%Y'))
loan_sub_kp['TIME_HISTORY'] = loan_sub_kp['ISSUE_D'] - loan_sub_kp['EARLIEST_CR_LINE']
loan_sub_kp['TIME_HISTORY'] = loan_sub_kp['TIME_HISTORY'].astype('timedelta64[D]').astype(int)

### 5.  Handle Null Values ...

In [9]:
# Note that in the output of describe, I have some columns with less than my 39999 rows.. this is due to NaN 
loan_sub_kp.describe()

,LOAN_AMNT,ANNUAL_INC,INQ_LAST_6MTHS,OPEN_ACC,PUB_REC,REVOL_UTIL,DTI,TOTAL_ACC,DELINQ_2YRS,MTHS_SINCE_LAST_DELINQ,DEFAULT,EMP_LISTED,EMPTY_DESC,EMP_NA,DELING_EVER,TIME_HISTORY
count,39999.00000,39999.000000,39999.000000,39999.000000,39999.000000,39949.000000,39999.000000,39999.000000,39999.000000,14158.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000
mean,11220.38176,69005.632504,0.889347,9.304558,0.055451,48.874830,13.328588,22.113228,0.147404,35.919551,0.142629,0.937973,0.331458,0.027176,0.353959,5037.440911
std,7458.32188,63903.736916,1.108814,4.414575,0.238177,28.331199,6.680936,11.419090,0.495918,22.016964,0.349698,0.241207,0.470743,0.162597,0.478203,2501.187961
min,500.00000,4000.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1095.000000
25%,5500.00000,40500.000000,0.000000,6.000000,0.000000,25.500000,8.190000,14.000000,0.000000,18.000000,0.000000,1.000000,0.000000,0.000000,0.000000,3348.000000
50%,10000.00000,59000.000000,1.000000,9.000000,0.000000,49.300000,13.420000,20.000000,0.000000,34.000000,0.000000,1.000000,0.000000,0.000000,0.000000,4595.000000
75%,15000.00000,82400.000000,1.000000,12.000000,0.000000,72.400000,18.620000,29.000000,0.000000,52.000000,0.000000,1.000000,1.000000,0.000000,1.000000,6239.000000
max,35000.00000,6000000.000000,8.000000,44.000000,4.000000,99.900000,29.990000,90.000000,11.000000,120.000000,1.000000,1.000000,1.000000,1.000000,1.000000,23892.000000


In [10]:
# Just Fill Nan values with mean from above ...
loan_sub_kp["REVOL_UTIL"].fillna(48.8, inplace=True)
loan_sub_kp["MTHS_SINCE_LAST_DELINQ"].fillna(35.9, inplace=True)

loan_sub_kp.describe()

,LOAN_AMNT,ANNUAL_INC,INQ_LAST_6MTHS,OPEN_ACC,PUB_REC,REVOL_UTIL,DTI,TOTAL_ACC,DELINQ_2YRS,MTHS_SINCE_LAST_DELINQ,DEFAULT,EMP_LISTED,EMPTY_DESC,EMP_NA,DELING_EVER,TIME_HISTORY
count,39999.00000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000,39999.000000
mean,11220.38176,69005.632504,0.889347,9.304558,0.055451,48.874736,13.328588,22.113228,0.147404,35.906920,0.142629,0.937973,0.331458,0.027176,0.353959,5037.440911
std,7458.32188,63903.736916,1.108814,4.414575,0.238177,28.313486,6.680936,11.419090,0.495918,13.098574,0.349698,0.241207,0.470743,0.162597,0.478203,2501.187961
min,500.00000,4000.000000,0.000000,2.000000,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1095.000000
25%,5500.00000,40500.000000,0.000000,6.000000,0.000000,25.500000,8.190000,14.000000,0.000000,35.900000,0.000000,1.000000,0.000000,0.000000,0.000000,3348.000000
50%,10000.00000,59000.000000,1.000000,9.000000,0.000000,49.300000,13.420000,20.000000,0.000000,35.900000,0.000000,1.000000,0.000000,0.000000,0.000000,4595.000000
75%,15000.00000,82400.000000,1.000000,12.000000,0.000000,72.400000,18.620000,29.000000,0.000000,35.900000,0.000000,1.000000,1.000000,0.000000,1.000000,6239.000000
max,35000.00000,6000000.000000,8.000000,44.000000,4.000000,99.900000,29.990000,90.000000,11.000000,120.000000,1.000000,1.000000,1.000000,1.000000,1.000000,23892.000000


## Save Cleaned Dataframe for Followon Modelling Phase

In [11]:
# Convert Pandas to spark dataframe ...

#spark_loan_df = spark.createDataFrame(loan)
#spark_loan_df.repartition(5).write.save("home/lending_club/loan", mode="overwrite")

spark_loan_sub_kp_df = spark.createDataFrame(loan_sub_kp)
spark_loan_sub_kp_df.repartition(5).write.save("home/lending_club/loan_sub_kp", mode="overwrite")

In [12]:
# Verify write with a read test
spark_df_test = spark.read.parquet("home/lending_club/loan_sub_kp")
spark_df_test.toPandas()

,LOAN_STATUS,ISSUE_D,LOAN_AMNT,EMP_TITLE,EMP_LENGTH,VERIFICATION_STATUS,HOME_OWNERSHIP,ANNUAL_INC,PURPOSE,INQ_LAST_6MTHS,DESC,OPEN_ACC,PUB_REC,REVOL_UTIL,DTI,TOTAL_ACC,DELINQ_2YRS,EARLIEST_CR_LINE,MTHS_SINCE_LAST_DELINQ,ADDR_STATE,TERM,DEFAULT,EMP_LISTED,EMPTY_DESC,EMP_NA,DELING_EVER,TIME_HISTORY
0,Fully Paid,1254355200000000000,18000,Automatic Data Processing,9 years,Source Verified,MORTGAGE,65000.00,small_business,1,552184 added on 10/07/09 > During times of ec...,15,0,49.4,9.16,34,0,891388800000000000,35.9,GA,36 months,0,1,0,0,0,4201
1,Fully Paid,1272672000000000000,15000,Palm Garden Port St. Lucie,8 years,Not Verified,MORTGAGE,78000.00,debt_consolidation,1,None,11,0,16.4,13.68,34,0,783648000000000000,35.9,FL,60 months,0,1,1,0,0,5660
2,Fully Paid,1272672000000000000,13750,"j. rogers architecture, inc.",8 years,Not Verified,MORTGAGE,33600.00,debt_consolidation,1,None,18,0,69.3,21.36,39,0,904608000000000000,61.0,MO,60 months,0,1,1,0,1,4260
3,Fully Paid,1272672000000000000,10000,DSD Laboratories,5 years,Not Verified,RENT,61152.00,major_purchase,0,Borrower added on 05/18/10 > Loan to pay for...,6,0,36.6,17.58,9,0,941414400000000000,35.9,WV,60 months,0,1,0,0,0,3834
4,Fully Paid,1272672000000000000,8000,Employee Incentive Plan,< 1 year,Not Verified,RENT,30000.00,debt_consolidation,0,Borrower added on 05/17/10 > Consolidating deb...,5,0,37.0,5.24,26,0,1001894400000000000,35.9,TX,60 months,0,1,0,0,0,3134
5,Fully Paid,1272672000000000000,7500,Feather Falls Casino,2 years,Not Verified,RENT,26400.00,moving,1,Borrower added on 05/17/10 > I am very excit...,6,0,24.1,13.41,11,0,978307200000000000,35.9,CA,60 months,0,1,0,0,0,3407
6,Fully Paid,1267401600000000000,9000,UPS,9 years,Verified,MORTGAGE,61000.00,debt_consolidation,1,Borrower added on 03/03/10 > This loan is to...,8,0,42.5,5.92,20,1,796694400000000000,20.0,KY,36 months,0,1,0,0,1,5448
7,Fully Paid,1304208000000000000,9200,US Army,4 years,Source Verified,RENT,23658.05,small_business,0,Borrower added on 05/11/11 > This loan will ...,2,0,71.1,12.88,4,0,1151712000000000000,35.9,CA,60 months,0,1,0,0,0,1765
8,Fully Paid,1304208000000000000,3900,Colorado Fondue Company,4 years,Source Verified,RENT,30000.00,credit_card,2,None,8,0,50.7,2.40,15,0,1088640000000000000,30.0,IL,36 months,0,1,1,0,1,2495
9,Current,1304208000000000000,35000,TA Instruments,3 years,Verified,MORTGAGE,67775.00,credit_card,1,Borrower added on 05/16/11 > I would like to c...,8,0,91.2,18.50,36,0,996624000000000000,53.0,DE,60 months,0,1,0,0,1,3560
